In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import chi2_contingency
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.multicomp import pairwise_tukeyhsd

# Define response and dependent variables
response_variables = ['Survival_5_years', 'Mortality']
dependent_variables = [
    'Country', 'Gender', 'Family_History', 'Smoking_History', 
    'Alcohol_Consumption', 'Obesity_BMI', 'Diet_Risk', 'Physical_Activity', 
    'Diabetes', 'Inflammatory_Bowel_Disease', 'Genetic_Mutation', 
    'Screening_History', 'Early_Detection', 'Treatment_Type', 
    'Urban_or_Rural', 'Economic_Classification', 'Healthcare_Access', 
    'Insurance_Status'
]

def load_data(file_path):
    """
    Load the dataset from a CSV file
    """
    try:
        df = pd.read_csv(file_path)
        print("Data loaded successfully with {} rows and {} columns".format(df.shape[0], df.shape[1]))
        return df
    except Exception as e:
        print("Error loading data: {}".format(e))
        return None

def check_missing_variables(df, response_vars, dependent_vars):
    """
    Check if all required variables exist in the dataframe
    """
    all_vars = response_vars + dependent_vars
    missing_vars = [var for var in all_vars if var not in df.columns]
    
    if missing_vars:
        print("Warning: The following variables are missing from the dataset: {}".format(missing_vars))
    return missing_vars

def perform_chi_square_tests(df, response_vars, dependent_vars):
    """
    Perform chi-square tests between each response and dependent variable pair
    """
    results = {}
    
    for response_var in response_vars:
        results[response_var] = {}
        
        for dependent_var in dependent_vars:
            # Skip if either variable is missing
            if response_var not in df.columns or dependent_var not in df.columns:
                results[response_var][dependent_var] = {
                    'chi2': None, 
                    'p': None, 
                    'dof': None,
                    'expected': None,
                    'contingency_table': None
                }
                continue
            
            # Create contingency table
            contingency_table = pd.crosstab(df[response_var], df[dependent_var])
            
            # Perform chi-square test
            chi2, p, dof, expected = chi2_contingency(contingency_table)
            
            # Store results
            results[response_var][dependent_var] = {
                'chi2': chi2,
                'p': p,
                'dof': dof,
                'expected': expected,
                'contingency_table': contingency_table
            }
    
    return results

def summarize_results(results, alpha=0.05):
    """
    Summarize the chi-square test results
    """
    summary = pd.DataFrame(columns=['Response_Variable', 'Dependent_Variable', 'Chi_Square', 'P_Value', 'DOF', 'Significant'])
    
    for response_var, dep_vars in results.items():
        for dep_var, stats in dep_vars.items():
            if stats['chi2'] is not None:
                row = {
                    'Response_Variable': response_var,
                    'Dependent_Variable': dep_var,
                    'Chi_Square': stats['chi2'],
                    'P_Value': stats['p'],
                    'DOF': stats['dof'],
                    'Significant': 'Yes' if stats['p'] < alpha else 'No'
                }
                summary = pd.concat([summary, pd.DataFrame([row])], ignore_index=True)
    
    # Sort by p-value to highlight the most significant relationships
    summary = summary.sort_values('P_Value')
    return summary

def plot_heatmap(summary, response_vars):
    """
    Create a heatmap of p-values for all variable combinations
    """
    # Reshape the data for heatmap
    pivot_data = summary.pivot(index='Dependent_Variable', columns='Response_Variable', values='P_Value')
    
    # Plot heatmap
    plt.figure(figsize=(10, 12))
    sns.heatmap(pivot_data, annot=True, cmap='viridis_r', vmin=0, vmax=0.05)
    plt.title('P-Values for Chi-Square Tests')
    plt.tight_layout()
    plt.savefig('chi_square_heatmap.png')
    plt.close()

def plot_significant_associations(df, results, alpha=0.05):
    """
    Create visualizations for significant associations
    """
    for response_var, dep_vars in results.items():
        significant_vars = [dep_var for dep_var, stats in dep_vars.items() 
                           if stats['p'] is not None and stats['p'] < alpha]
        
        for dep_var in significant_vars[:5]:  # Plot top 5 significant associations
            # Skip if data is missing
            if response_var not in df.columns or dep_var not in df.columns:
                continue
                
            # Create a stacked bar chart
            plt.figure(figsize=(10, 6))
            
            # Get contingency table
            cont_table = results[response_var][dep_var]['contingency_table']
            cont_table_pct = cont_table.div(cont_table.sum(axis=1), axis=0) * 100
            
            # Plot
            cont_table_pct.plot(kind='bar', stacked=True)
            plt.title('Association between {} and {}'.format(response_var, dep_var))
            plt.xlabel(response_var)
            plt.ylabel('Percentage')
            plt.xticks(rotation=45)
            plt.legend(title=dep_var)
            plt.tight_layout()
            plt.savefig('{}_{}_association.png'.format(response_var, dep_var))
            plt.close()

def main():
    # Your actual data file path
    file_path = r'C:\Users\Aru\Downloads\archive (1)\colorectal_cancer_dataset.csv'
    
    # Load data
    df = load_data(file_path)
    if df is None:
        return
    
    # Check for missing variables
    missing_vars = check_missing_variables(df, response_variables, dependent_variables)
    if set(response_variables).issubset(set(missing_vars)):
        print("Error: All response variables are missing. Cannot proceed.")
        return
    
    # Filter out missing variables
    available_dep_vars = [var for var in dependent_variables if var not in missing_vars]
    available_resp_vars = [var for var in response_variables if var not in missing_vars]
    
    # Perform chi-square tests
    results = perform_chi_square_tests(df, available_resp_vars, available_dep_vars)
    
    # Summarize results
    summary = summarize_results(results)
    print("\nChi-square Test Results Summary:")
    print(summary)
    
    # Save results to CSV
    summary.to_csv('chi_square_results.csv', index=False)
    print("Results saved to chi_square_results.csv")
    
    # Create visualizations
    if not summary.empty:
        plot_heatmap(summary, available_resp_vars)
        plot_significant_associations(df, results)
        print("Visualizations created and saved.")

# For demonstration purposes, here's how you would create a sample dataset
def create_sample_dataset(n_samples=1000):
    """
    Create a sample dataset with the specified variables for demonstration
    """
    np.random.seed(42)
    
    # Generate random categorical data
    data = {
        'Survival_5_years': np.random.choice(['Yes', 'No'], size=n_samples, p=[0.7, 0.3]),
        'Mortality': np.random.choice(['Yes', 'No'], size=n_samples, p=[0.25, 0.75]),
        'Country': np.random.choice(['USA', 'UK', 'Canada', 'Australia', 'Germany'], size=n_samples),
        'Gender': np.random.choice(['Male', 'Female'], size=n_samples),
        'Family_History': np.random.choice(['Yes', 'No'], size=n_samples, p=[0.3, 0.7]),
        'Smoking_History': np.random.choice(['Never', 'Former', 'Current'], size=n_samples, p=[0.5, 0.3, 0.2]),
        'Alcohol_Consumption': np.random.choice(['None', 'Moderate', 'Heavy'], size=n_samples),
        'Obesity_BMI': np.random.choice(['Normal', 'Overweight', 'Obese'], size=n_samples),
        'Diet_Risk': np.random.choice(['Low', 'Medium', 'High'], size=n_samples),
        'Physical_Activity': np.random.choice(['Sedentary', 'Moderate', 'Active'], size=n_samples),
        'Diabetes': np.random.choice(['Yes', 'No'], size=n_samples, p=[0.15, 0.85]),
        'Inflammatory_Bowel_Disease': np.random.choice(['Yes', 'No'], size=n_samples, p=[0.05, 0.95]),
        'Genetic_Mutation': np.random.choice(['Yes', 'No'], size=n_samples, p=[0.1, 0.9]),
        'Screening_History': np.random.choice(['Regular', 'Irregular', 'None'], size=n_samples),
        'Early_Detection': np.random.choice(['Yes', 'No'], size=n_samples),
        'Treatment_Type': np.random.choice(['Surgery', 'Chemotherapy', 'Radiation', 'Combined', 'None'], size=n_samples),
        'Urban_or_Rural': np.random.choice(['Urban', 'Suburban', 'Rural'], size=n_samples),
        'Economic_Classification': np.random.choice(['Low', 'Middle', 'High'], size=n_samples),
        'Healthcare_Access': np.random.choice(['Good', 'Limited', 'Poor'], size=n_samples),
        'Insurance_Status': np.random.choice(['Insured', 'Underinsured', 'Uninsured'], size=n_samples)
    }
    
    # Create associations to make the data more realistic
    # Make smoking affect survival
    for i in range(n_samples):
        if data['Smoking_History'][i] == 'Current':
            # Current smokers have worse survival odds
            if np.random.random() < 0.5:  # 50% chance to override
                data['Survival_5_years'][i] = 'No'
                data['Mortality'][i] = 'Yes'
        
        # Make treatment type affect survival
        if data['Treatment_Type'][i] == 'None':
            # No treatment has worse outcomes
            if np.random.random() < 0.7:  # 70% chance to override
                data['Survival_5_years'][i] = 'No'
                data['Mortality'][i] = 'Yes'
        
        # Make healthcare access affect survival
        if data['Healthcare_Access'][i] == 'Poor':
            # Poor healthcare access means worse outcomes
            if np.random.random() < 0.6:  # 60% chance to override
                data['Survival_5_years'][i] = 'No'
    
    return pd.DataFrame(data)

if __name__ == "__main__":
    # Uncomment to use sample data if you don't have a real dataset
    # sample_df = create_sample_dataset()
    # sample_df.to_csv('your_data.csv', index=False)
    # print("Sample dataset created and saved to your_data.csv")
    
    main()

Data loaded successfully with 167497 rows and 28 columns


C:\Users\Aru\AppData\Local\Temp\ipykernel_13924\1456581346.py:99: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  summary = pd.concat([summary, pd.DataFrame([row])], ignore_index=True)



Chi-square Test Results Summary:
   Response_Variable          Dependent_Variable  Chi_Square   P_Value DOF  \
2   Survival_5_years              Family_History    4.044551  0.044314   1   
25         Mortality           Physical_Activity    4.313122  0.115722   2   
4   Survival_5_years         Alcohol_Consumption    2.149824  0.142586   1   
11  Survival_5_years           Screening_History    3.641267  0.161923   2   
17  Survival_5_years            Insurance_Status    1.789104  0.181035   1   
14  Survival_5_years              Urban_or_Rural    1.618743  0.203267   1   
19         Mortality                      Gender    1.066354  0.301770   1   
28         Mortality            Genetic_Mutation    1.021965  0.312053   1   
6   Survival_5_years                   Diet_Risk    2.107042  0.348708   2   
27         Mortality  Inflammatory_Bowel_Disease    0.592265  0.441544   1   
18         Mortality                     Country   14.598673  0.480693  15   
29         Mortality          

<Figure size 1000x600 with 0 Axes>

The Multiple Chi Squares test was chosen due to multiple categorical variables. 
The only categorical variable which was a significant predictor for 5 year global colorectal cancer survival was family history. 
Nonsignificance of most variables does not necessarily predict clinical significance. 
This study must be repeated numerous times to come to a conclusion in regards to global colorectal cancer 5 year survival. 

In [16]:
# Import necessary libraries
import pandas as pd
import numpy as np
from scipy.stats import kruskal
import matplotlib.pyplot as plt
import seaborn as sns

# Define file path
file_path = "C:\\Users\\Aru\\Downloads\\archive (1)\\colorectal_cancer_dataset.csv"

# Load the data
print("Loading data...")
try:
    df = pd.read_csv(file_path)
    print("Data loaded successfully with {} rows and {} columns".format(df.shape[0], df.shape[1]))
except Exception as e:
    print("Error loading data: {}".format(e))
    exit()

# Define response and dependent variables
response_variables = ['Incidence_Rate_per_100k', 'Mortality_Rate_per_100k']
dependent_variables = [
    'Country', 'Gender', 'Family_History', 'Smoking_History', 
    'Alcohol_Consumption', 'Obesity_BMI', 'Diet_Risk', 'Physical_Activity', 
    'Diabetes', 'Inflammatory_Bowel_Disease', 'Genetic_Mutation', 
    'Screening_History', 'Early_Detection', 'Treatment_Type', 
    'Urban_or_Rural', 'Economic_Classification', 'Healthcare_Access', 
    'Insurance_Status'
]

# Check if response variables exist, create them if they don't
# This is just for demonstration - you may need to adjust how these are calculated
for var in response_variables:
    if var not in df.columns:
        print("Creating sample {} variable for demonstration".format(var))
        if var == 'Incidence_Rate_per_100k':
            df[var] = np.random.normal(50, 15, size=len(df))
        elif var == 'Mortality_Rate_per_100k':
            df[var] = np.random.normal(25, 10, size=len(df))

# Create a results dataframe
results = pd.DataFrame(columns=['Response_Variable', 'Dependent_Variable', 
                               'Statistic', 'P_Value', 'Significant'])

# Perform Kruskal-Wallis tests
print("Performing Kruskal-Wallis tests...")
for response_var in response_variables:
    for dependent_var in dependent_variables:
        # Skip if either variable is missing
        if response_var not in df.columns or dependent_var not in df.columns:
            print("Skipping {} vs {} - variables not found".format(response_var, dependent_var))
            continue
        
        # Get unique categories in the dependent variable
        categories = df[dependent_var].unique()
        
        # Skip if less than 2 categories
        if len(categories) < 2:
            print("Skipping {} vs {} - less than 2 categories".format(response_var, dependent_var))
            continue
        
        # Prepare samples for Kruskal-Wallis test
        samples = []
        
        for category in categories:
            category_data = df[df[dependent_var] == category][response_var].dropna()
            
            # Skip categories with less than 3 samples
            if len(category_data) < 3:
                continue
            
            samples.append(category_data)
        
        # Skip if less than 2 valid categories remain
        if len(samples) < 2:
            print("Skipping {} vs {} - insufficient data".format(response_var, dependent_var))
            continue
        
        # Perform Kruskal-Wallis test
        try:
            # Unpack samples for the kruskal function
            statistic, p_value = kruskal(*samples)
            
            # Add to results
            new_row = {
                'Response_Variable': response_var,
                'Dependent_Variable': dependent_var,
                'Statistic': statistic,
                'P_Value': p_value,
                'Significant': 'Yes' if p_value < 0.05 else 'No'
            }
            
            results = pd.concat([results, pd.DataFrame([new_row])], ignore_index=True)
            
            print("Test for {} vs {}: p = {:.4f}".format(response_var, dependent_var, p_value))
            
        except Exception as e:
            print("Error in test for {} vs {}: {}".format(response_var, dependent_var, e))

# Sort results by p-value
results = results.sort_values('P_Value')

# Display results
print("\nKruskal-Wallis Test Results:")
print(results)

# Save results to CSV
results.to_csv('kruskal_wallis_results.csv', index=False)
print("Results saved to kruskal_wallis_results.csv")

# Plot significant results
print("\nCreating plots for significant relationships...")
for index, row in results[results['Significant'] == 'Yes'].head(5).iterrows():
    response_var = row['Response_Variable']
    dependent_var = row['Dependent_Variable']
    
    # Create boxplot
    plt.figure(figsize=(10, 6))
    sns.boxplot(x=dependent_var, y=response_var, data=df)
    plt.title('Distribution of {} by {} (p={:.4f})'.format(
        response_var, dependent_var, row['P_Value']))
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.savefig('{}_{}_boxplot.png'.format(response_var, dependent_var))
    plt.close()
    print("Created plot for {} vs {}".format(response_var, dependent_var))

print("\nAnalysis complete!")

Loading data...
Data loaded successfully with 167497 rows and 28 columns
Creating sample Incidence_Rate_per_100k variable for demonstration
Creating sample Mortality_Rate_per_100k variable for demonstration
Performing Kruskal-Wallis tests...


C:\Users\Aru\AppData\Local\Temp\ipykernel_13924\3423131837.py:93: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results = pd.concat([results, pd.DataFrame([new_row])], ignore_index=True)


Test for Incidence_Rate_per_100k vs Country: p = 0.6286
Test for Incidence_Rate_per_100k vs Gender: p = 0.9921
Test for Incidence_Rate_per_100k vs Family_History: p = 0.8715
Test for Incidence_Rate_per_100k vs Smoking_History: p = 0.9079
Test for Incidence_Rate_per_100k vs Alcohol_Consumption: p = 0.0532
Test for Incidence_Rate_per_100k vs Obesity_BMI: p = 0.5344
Test for Incidence_Rate_per_100k vs Diet_Risk: p = 0.1064
Test for Incidence_Rate_per_100k vs Physical_Activity: p = 0.3140
Test for Incidence_Rate_per_100k vs Diabetes: p = 0.0525
Test for Incidence_Rate_per_100k vs Inflammatory_Bowel_Disease: p = 0.8975
Test for Incidence_Rate_per_100k vs Genetic_Mutation: p = 0.3976
Test for Incidence_Rate_per_100k vs Screening_History: p = 0.6277
Test for Incidence_Rate_per_100k vs Early_Detection: p = 0.9079
Test for Incidence_Rate_per_100k vs Treatment_Type: p = 0.7911
Test for Incidence_Rate_per_100k vs Urban_or_Rural: p = 0.6483
Test for Incidence_Rate_per_100k vs Economic_Classificati

A subsequent kruskall-wallis test was used to determine which categorical variables were the best predictors of mortality and incidence rates (per 100k) of global colorectal cancer. 
Early detection and Urban v Rural living were significant predictors of mortality rates per 100k of global colorectal cancer rates. 
The following categorical variables were not deemed significant predictors of mortality and incidence rates, but were nonetheless close: Diabetes and Alcohol Consumption (IR), and Gender, Family History, and Healthcare Access. 
These may be effect modifiers; not things that effect rates directly but can impact outcomes in concert with the aforementioned statistically significant variables. 
This deeper analysis also shows that gender is still in consideration when examining mortality rates; the prior analysis showed that gender may also affect mortality deaths in aboslute terms. 

In [17]:
# Import necessary libraries
import pandas as pd
import numpy as np
from scipy.stats import spearmanr
import matplotlib.pyplot as plt
import seaborn as sns

# Define file path directly - no function parameters with path
file_path = "C:\\Users\\Aru\\Downloads\\archive (1)\\colorectal_cancer_dataset.csv"

# Define variables for analysis
response_variables = ['Mortality', 'Incidence_per_100k', 'Mortality_rate_per_100k']
dependent_variables = ['Healthcare_Costs', 'Age']

# Load the data
print("Loading data from:", file_path)
try:
    df = pd.read_csv(file_path)
    print("Data loaded successfully with {} rows and {} columns".format(df.shape[0], df.shape[1]))
    
    # Display the first few rows to verify data
    print("\nFirst 5 rows of the dataset:")
    print(df.head())
    
    # Display column names to check for variable names
    print("\nAvailable columns in the dataset:")
    print(df.columns.tolist())
except Exception as e:
    print("Error loading data: {}".format(e))
    # Create sample data for demonstration if file loading fails
    print("Creating sample dataset for demonstration...")
    np.random.seed(42)
    n_samples = 100
    df = pd.DataFrame({
        'Mortality': np.random.choice(['Yes', 'No'], size=n_samples),
        'Incidence_per_100k': np.random.normal(50, 15, size=n_samples),
        'Mortality_rate_per_100k': np.random.normal(25, 8, size=n_samples),
        'Healthcare_Costs': np.random.normal(5000, 2000, size=n_samples),
        'Age': np.random.normal(65, 12, size=n_samples)
    })
    print("Sample data created successfully")

# Check for missing variables and convert categorical to numeric if needed
print("\nChecking variables...")
for var in response_variables + dependent_variables:
    if var not in df.columns:
        print("Warning: {} not found in dataset. Creating sample data.".format(var))
        if var == 'Mortality':
            df[var] = np.random.choice(['Yes', 'No'], size=len(df))
        elif var == 'Incidence_per_100k':
            df[var] = np.random.normal(50, 15, size=len(df))
        elif var == 'Mortality_rate_per_100k':
            df[var] = np.random.normal(25, 8, size=len(df))
        elif var == 'Healthcare_Costs':
            df[var] = np.random.normal(5000, 2000, size=len(df))
        elif var == 'Age':
            df[var] = np.random.normal(65, 12, size=len(df))
    
    # Check if variable is categorical and needs conversion for correlation
    if var == 'Mortality' and var in df.columns:
        if df[var].dtype == 'object':
            print("Converting categorical '{}' to numeric for correlation analysis".format(var))
            # Map Yes to 1, No to 0
            df[var + '_numeric'] = df[var].map({'Yes': 1, 'No': 0})
            # Replace Mortality with the numeric version in response variables
            response_variables[response_variables.index('Mortality')] = 'Mortality_numeric'

# Create a results dataframe
results = pd.DataFrame(columns=['Response_Variable', 'Dependent_Variable', 
                                'Correlation_Coefficient', 'P_Value', 'Significant'])

# Perform Spearman rank correlation tests
print("\nPerforming Spearman rank correlation tests...")
for response_var in response_variables:
    for dependent_var in dependent_variables:
        # Skip if either variable is missing
        if response_var not in df.columns or dependent_var not in df.columns:
            print("Skipping {} vs {} - variables not found".format(response_var, dependent_var))
            continue
        
        # Get data for correlation
        x = df[dependent_var].dropna()
        y = df[response_var].dropna()
        
        # Ensure same length by using common indices
        common_indices = x.index.intersection(y.index)
        x = x.loc[common_indices]
        y = y.loc[common_indices]
        
        # Skip if not enough data
        if len(x) < 5:
            print("Skipping {} vs {} - insufficient data".format(response_var, dependent_var))
            continue
        
        # Perform Spearman correlation
        try:
            correlation, p_value = spearmanr(x, y)
            
            # Add to results
            new_row = {
                'Response_Variable': response_var,
                'Dependent_Variable': dependent_var,
                'Correlation_Coefficient': correlation,
                'P_Value': p_value,
                'Significant': 'Yes' if p_value < 0.05 else 'No'
            }
            
            results = pd.concat([results, pd.DataFrame([new_row])], ignore_index=True)
            
            print("Correlation for {} vs {}: rho = {:.3f}, p = {:.4f}".format(
                response_var, dependent_var, correlation, p_value))
            
            # Create scatter plot
            plt.figure(figsize=(10, 6))
            sns.scatterplot(x=df[dependent_var], y=df[response_var])
            
            # Add regression line
            sns.regplot(x=df[dependent_var], y=df[response_var], scatter=False, 
                       line_kws={"color": "red"})
            
            plt.title('Spearman Correlation: {} vs {}\nrho={:.3f}, p={:.4f}'.format(
                response_var, dependent_var, correlation, p_value))
            plt.xlabel(dependent_var)
            plt.ylabel(response_var)
            plt.tight_layout()
            plt.savefig('{}_{}_correlation.png'.format(response_var, dependent_var))
            plt.close()
            
        except Exception as e:
            print("Error in correlation for {} vs {}: {}".format(response_var, dependent_var, e))

# Display interpretation of results
print("\n===== SPEARMAN CORRELATION RESULTS =====")
print(results)

# Save results to CSV
results.to_csv('spearman_correlation_results.csv', index=False)
print("\nResults saved to spearman_correlation_results.csv")

# Create a correlation heatmap
print("\nCreating correlation heatmap...")
try:
    # Get all numeric variables
    numeric_vars = response_variables + dependent_variables
    
    # Create correlation matrix
    corr_matrix = df[numeric_vars].corr(method='spearman')
    
    # Plot heatmap
    plt.figure(figsize=(10, 8))
    sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', vmin=-1, vmax=1, 
               fmt='.2f', linewidths=0.5)
    plt.title('Spearman Rank Correlation Heatmap')
    plt.tight_layout()
    plt.savefig('spearman_correlation_heatmap.png')
    plt.close()
    print("Correlation heatmap saved as spearman_correlation_heatmap.png")
except Exception as e:
    print("Error creating heatmap:", e)

print("\nAnalysis complete!")

# Print interpretation guidelines
print("\n===== INTERPRETATION GUIDELINES =====")
print("Spearman's rho interpretation:")
print("0.00 to 0.19: 'very weak' correlation")
print("0.20 to 0.39: 'weak' correlation")
print("0.40 to 0.59: 'moderate' correlation")
print("0.60 to 0.79: 'strong' correlation")
print("0.80 to 1.00: 'very strong' correlation")
print("\nNegative values indicate inverse relationships")
print("P-value < 0.05 indicates statistical significance")

Loading data from: C:\Users\Aru\Downloads\archive (1)\colorectal_cancer_dataset.csv
Data loaded successfully with 167497 rows and 28 columns

First 5 rows of the dataset:
   Patient_ID Country  Age Gender Cancer_Stage  Tumor_Size_mm Family_History  \
0           1      UK   77      M    Localized             69             No   
1           2      UK   59      M    Localized             33             No   
2           3   Japan   66      M     Regional             17             No   
3           4     USA   83      M     Regional             14             No   
4           5  France   66      M    Localized             34             No   

  Smoking_History Alcohol_Consumption Obesity_BMI  ... Survival_5_years  \
0              No                 Yes  Overweight  ...              Yes   
1              No                  No  Overweight  ...              Yes   
2             Yes                  No      Normal  ...              Yes   
3              No                  No       Obes

C:\Users\Aru\AppData\Local\Temp\ipykernel_13924\2437114217.py:108: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results = pd.concat([results, pd.DataFrame([new_row])], ignore_index=True)


Correlation for Mortality_numeric vs Age: rho = 0.000, p = 0.9209
Correlation for Incidence_per_100k vs Healthcare_Costs: rho = 0.003, p = 0.2592
Correlation for Incidence_per_100k vs Age: rho = -0.001, p = 0.5570
Correlation for Mortality_rate_per_100k vs Healthcare_Costs: rho = 0.003, p = 0.2486
Correlation for Mortality_rate_per_100k vs Age: rho = -0.005, p = 0.0410

===== SPEARMAN CORRELATION RESULTS =====
         Response_Variable Dependent_Variable  Correlation_Coefficient  \
0        Mortality_numeric   Healthcare_Costs                -0.003392   
1        Mortality_numeric                Age                 0.000243   
2       Incidence_per_100k   Healthcare_Costs                 0.002757   
3       Incidence_per_100k                Age                -0.001435   
4  Mortality_rate_per_100k   Healthcare_Costs                 0.002819   
5  Mortality_rate_per_100k                Age                -0.004993   

    P_Value Significant  
0  0.165043          No  
1  0.920943    

This spearman rank correlation coefficient test was chosen due to there being two continuous numerical variables measured one at a time. Age was deemed the only statistically significant variable, with a weak very weak negative correlation. 